In [1]:
import whisper

In [3]:
model = whisper.load_model("large-v3")

100%|█████████████████████████████████████| 2.88G/2.88G [00:36<00:00, 83.5MiB/s]


In [4]:
TESTF = "/home/joregan/Listening_test/00_4.wav"

In [35]:
audio = whisper.load_audio(TESTF)
audio = whisper.pad_or_trim(audio)

In [36]:
mel = whisper.log_mel_spectrogram(audio, n_mels=128).to(model.device)

In [37]:
language_tokens, probs = model.detect_language(mel)

In [33]:
language_tokens

tensor(50273, device='cuda:0')

In [16]:
print("Swedish", probs['sv'], "English", probs["en"])

Swedish 0.9028066992759705 English 0.03867207467556


In [19]:
from whisper.decoding import detect_language

In [ ]:
model

In [ ]:
detect_language("Whisper", mel=mel)